# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine


Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
driver = "mysql+pymysql"
user = "ironhacker_read"
password = "ir0nhack3r"
ip = "35.239.232.23"
database = "orders"

connection_string = f"{driver}://{user}:{password}@{ip}/{database}"
    
engine = create_engine(connection_string)

query = """
        SELECT * FROM orders
"""

orders = pd.read_sql(query, engine)

orders.to_csv("/home/chao/Desktop/Ironhack/Lab/lab-df-calculation-and-transformation/orders.csv", sep = ",")

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [24]:
# your code here
orders = pd.read_csv("../orders.csv", sep = ",")
orders = orders.drop(["Unnamed: 0", "index"] , axis=1)
orders_spend_sum = orders.groupby(["CustomerID"]).sum()
percentil95 = orders_spend_sum["amount_spent"].quantile(0.95)
percentil75 = orders_spend_sum["amount_spent"].quantile(0.75)
orders_spend_sum["customer_type"] = orders_spend_sum["amount_spent"].apply(lambda x:  (x >= percentil95 and "VIP") or (x < percentil75 and "Normal") or "Preferred")
orders_spend_sum.head()

,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,customer_type
CustomerID,,,,,,,,,
12346,541431,2011,1,2,10,74215,1.04,77183.60,VIP
12347,101296926,365971,1383,441,2219,2458,481.21,4310.00,Preferred
12348,16869685,62324,257,111,472,2341,178.71,1797.24,Preferred
12349,42165457,146803,803,73,657,631,605.10,1757.55,Preferred
12350,9231629,34187,34,51,272,197,65.30,334.40,Normal


In [58]:
orders_spend_sum["customer_type"].value_counts()

Normal       3254
Preferred     868
VIP           217
Name: customer_type, dtype: int64

In [60]:
orders_spend_sum.shape

(4339, 9)

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [25]:
# your code here
orders_spend_type = orders_spend_sum[["customer_type"]]
orders_new = pd.merge(orders,orders_spend_type, how = "inner", on=["CustomerID"])
orders_new.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,customer_type
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


In [83]:
orders_customer = pd.DataFrame(orders_new.groupby(["customer_type", "Country"])["CustomerID"].nunique())
orders_customer= orders_customer.rename(columns={"CustomerID":"count"})
orders_customer.loc["VIP"]

,count
Country,
Australia,2
Belgium,1
Channel Islands,1
Cyprus,1
Denmark,1
EIRE,2
Finland,1
France,9
Germany,10


In [ ]:
# United Kingdom is the country that has the most VIP customers

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [81]:
orders_VIP = orders_customer.loc["VIP"]
orders_Preferred = orders_customer.loc["Preferred"]


orders_VIP_Preferred = pd.merge(orders_VIP,orders_Preferred, left_index=True, right_index=True)
orders_VIP_Preferred["total"] = orders_VIP_Preferred["count_x"] + orders_VIP_Preferred["count_y"]
orders_VIP_Preferred

,count_x,count_y,total
Country,,,
Australia,2,2,4
Belgium,1,11,12
Channel Islands,1,3,4
Cyprus,1,3,4
Denmark,1,2,3
EIRE,2,1,3
Finland,1,4,5
France,9,20,29
Germany,10,29,39


In [ ]:
# United Kingdom is still the country where has most VIP and Preferred customers